In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
#import jieba
import re

In [2]:
import matplotlib.pyplot as plt 
import seaborn as sns
import glob
import os 
from tqdm.auto import tqdm
import re

# Load file data:

In [3]:
use_cols = ['product_id','sub_category','price','rating','clean_desc','product_name','clean_prd_name']
clean_prd_df = pd.read_csv('../DATA/final_details.csv',usecols=use_cols)

In [4]:
clean_prd_df.head()

,product_id,product_name,sub_category,price,rating,clean_desc,clean_prd_name
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Áo Ba Lỗ,86250.0,4.9,áo lỗ chiều đường_phố nhiệt_đới tal fit xuất_x...,áo lỗ thun gân form body
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Áo Ba Lỗ,26800.0,4.9,áo lỗ xuất_xứ việt_nam tổ_chức trách_nhiệm sản...,áo lỗ nam trắng chất_cotton siêu_mát đẹp
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Áo Ba Lỗ,39500.0,4.8,áo lỗ thương_hiệu tyasuo chiều áo không_thể đư...,áo lỗ nam tyasuo chất vải co_dãn mát không_xù mềm
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,16500.0,4.8,áo lỗ chất_liệu hàng gửi hà_nội áo lỗ nam mặc ...,áo lỗ hàng việt_nam
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Ba Lỗ,45000.0,4.8,áo lỗ chiều áo không_thể hàng gửi hà_nội thông...,áo thun nam thể_thao lỗ mẫu mới đẹp


## Check na values from clean data

In [6]:
#clean_prd_df[clean_prd_df['clean_prd_name'].apply(lambda x: isinstance(x,float))]

In [5]:
drop_idx = clean_prd_df[clean_prd_df['clean_prd_name'].apply(lambda x: isinstance(x,float))].index
drop_idx

Int64Index([], dtype='int64')

In [6]:
clean_prd_df = clean_prd_df.drop(index=drop_idx)

In [7]:
def export_dict_gensim(col_values: np.array):
    # Tokenize(split) the sentences into words
    #products_gem = [ for x in col_values]
    products_gem = list()
    for x in tqdm(col_values,"Tokenize data"):
        if isinstance(x,str):
            split_txt = [text for text in x.split()]
            products_gem.append(split_txt)
            
    # remove some special elements in texts
    # number
    products_gem_re = [[re.sub('[0-9]+', '', e) for e in text] for text in products_gem]
    # special symbols
    special_ls = ['', ' ', ',', '.', '...', '-', ':', ';', '?',
                  '%', '_%', '(', ')', '+', '/', 'g', 'ml']
    products_gem_re = [[t.lower() for t in text if t not in special_ls] for text in products_gem_re]

    # Obtain the number of features based on dictionary: Use corpora.Dictionary
    dictionary = corpora.Dictionary(products_gem_re)
    print('Numbers of features in **Dictionary**:',len(dictionary.token2id))

    # Obtain corpus based on dictionary (dense matrix)
    corpus = [dictionary.doc2bow(text) for text in products_gem_re]
    return dictionary, corpus

In [8]:
prd_name_values = clean_prd_df['clean_prd_name'].values

In [9]:
name_prd_dict, name_prd_corpus = export_dict_gensim(col_values=prd_name_values)

Tokenize data:   0%|          | 0/49594 [00:00<?, ?it/s]

Numbers of features in **Dictionary**: 21589


- View the sample of words from Gensim

In [10]:
list(name_prd_dict.token2id.keys())[0:10]

['body',
 'form',
 'gân',
 'lỗ',
 'thun',
 'áo',
 'chất_cotton',
 'nam',
 'siêu_mát',
 'trắng']

In [11]:
name_prd_corpus[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]

# Apply model TFIDF:

In [12]:
# Create the TF-IDF model
tfidf = models.TfidfModel(corpus=name_prd_corpus, smartirs='ntc')

In [13]:
# Show the TF-IDF weights
for doc in tfidf[name_prd_corpus][0:10]:
    print([[name_prd_dict[id], np.around(freq, decimals=2)] for id, freq in doc])
    print("\n\n")

[['body', 0.49], ['form', 0.34], ['gân', 0.68], ['lỗ', 0.35], ['thun', 0.23], ['áo', 0.13]]



[['lỗ', 0.35], ['áo', 0.13], ['chất_cotton', 0.57], ['nam', 0.08], ['siêu_mát', 0.58], ['trắng', 0.36], ['đẹp', 0.26]]



[['lỗ', 0.26], ['áo', 0.09], ['nam', 0.06], ['chất', 0.21], ['co_dãn', 0.3], ['không_xù', 0.43], ['mát', 0.22], ['mềm', 0.28], ['tyasuo', 0.67], ['vải', 0.19]]



[['lỗ', 0.49], ['áo', 0.18], ['hàng', 0.35], ['việt_nam', 0.78]]



[['lỗ', 0.48], ['thun', 0.31], ['áo', 0.17], ['nam', 0.1], ['đẹp', 0.35], ['mẫu', 0.49], ['mới', 0.42], ['thể_thao', 0.3]]



[['lỗ', 0.19], ['áo', 0.07], ['nam', 0.04], ['siêu_mát', 0.31], ['thể_thao', 0.12], ['chất_liệu', 0.17], ['co_giãn', 0.17], ['cotton', 0.15], ['hút', 0.22], ['kĩ_hàng', 0.49], ['may', 0.29], ['mồ_hôi', 0.24], ['sát_nách', 0.42], ['thấm', 0.22], ['đường', 0.33]]



[['lỗ', 0.31], ['áo', 0.11], ['nam', 0.03], ['mềm', 0.17], ['co_giãn', 0.14], ['sát_nách', 0.34], ['cao_cấp', 0.09], ['gym', 0.19], ['kiểu', 0.19], ['lót_form', 

In [14]:
# tính toán sự tương tự trong ma trận thưa thớt
feature_cnt = len(name_prd_dict.token2id)
index = similarities.SparseMatrixSimilarity(tfidf[name_prd_corpus],
                                            num_features = feature_cnt)

In [20]:
#index[tfidf[name_prd_corpus]][0]

In [15]:
index[tfidf[name_prd_corpus[0]]]

array([1.        , 0.13861008, 0.10136303, ..., 0.        , 0.        ,
       0.        ], dtype=float32)

In [16]:
type(index)

gensim.similarities.docsim.SparseMatrixSimilarity

# Save Data:

In [17]:
type(tfidf)

gensim.models.tfidfmodel.TfidfModel

In [18]:
tfidf.save('../DATA/Gensim/tfidf_model')

In [19]:
name_prd_dict.save('../DATA/Gensim/corpus_dictionary')

In [20]:
index.save("../DATA/Gensim/similarity_matrix.index")

In [1]:
#data_result = index[tfidf[name_prd_corpus]]